# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)
#### Aluno: Bruno Crespo Ferreira

## Problema

Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é de R\$ 2,00 nas duas primeiras semanas e R\$ 2,50 nas duas últimas, quando a concorrência demandar por material no mercado. A demanda semanal de camisetas será de 5.000, 10.000, 30.000 e 60.000. A capacidade máxima de produção da empresa é de 25.000 camisetas semanalmente. Na primeira e na segunda semanas, a empresa poderá contratar horas extras de serviço e fabricar mais 10.000 camisetas em cada semana. Nesse caso, o custo de produção sobe para R\$ 2,80. O excesso de produção pode ser estocado a um custo de R\$ 0,20 por unidade por semana.

### Objetivo

Formule um modelo que minimize os custos.

## Instalação da biblioteca PuLP

In [1]:
!pip install pulp
import pulp

## Parâmetros

* $T$: conjunto de períodos.
* $c_{t} \in \mathbb{R}_+$: custo de produção de cada camisa no período $t \in T$.
* $d_{t} \in \mathbb{N}$: demanda de camisetas no período $t \in T$.
* $capacidade\_maxima \in \mathbb{N}$: capacidade máxima de produção de camisetas semanalmente.
* $h_{t} \in \mathbb{R}_+$: custo de estocagem de cada camisa no período $t \in \{1, 2, 3\}$.

Observação: $c_{t}$ possui variações considerando a possibilidade de haver horas extras, representado como $c_{t*}$.

In [2]:
periodos = [1, 2, 3, 4]

periodos_horas_extras = [1, 2]

custos_de_producao = {
    1: 2.00,
    2: 2.00,
    3: 2.50,
    4: 2.50
}

custos_de_producao_horas_extras = {
    1: 2.80,
    2: 2.80
}

demanda = {
    1: 5000,
    2: 10000,
    3: 30000,
    4: 60000
}

capacidade_maxima = 25000

custo_de_estoque = {
    1: 0.20,
    2: 0.20,
    3: 0.20
}

## Declaração do objeto que representa o modelo matemático

Inicialmente, a ideia era criar dois modelos: um que considera não considera as horas extras e outro que considera.

Considerando essa implementação várias partes do código ficaram repetitivas, portanto decidiu-se utilizar uma pulp.LpVariable do tipo Binary para representar as horas extras e outras modificações...

In [3]:
modelo = pulp.LpProblem('fabrica_de_camisetas', pulp.LpMinimize)

## Variáveis de decisão

* $x_{t} \ge 0$: quantidade de camisetas produzidas no período $t \in T$ (sem horas extras).
* $y_{t} \ge 0$: quantidade de camisetas produzidas no período $t \in \{1, 2\}$ (usando horas extras).
* $I_{t} \ge 0$: quantidade de camisetas estocadas no fim do perído $t \in T$.
* $b_{t}$: variável binária indicando se as horas extras foram consideradas no período $t \in \{1, 2\}$

In [4]:
producao_normal_vars = pulp.LpVariable.dicts('producao', periodos, cat='Integer', lowBound=0)
producao_horas_extras_vars = pulp.LpVariable.dicts('producao_horas_extras', periodos_horas_extras, cat='Integer', lowBound=0)
estoque_vars = pulp.LpVariable.dicts('estoque', periodos, cat='Integer', lowBound=0)
tem_horas_extras = pulp.LpVariable.dicts('tem_horas_extras', periodos_horas_extras, cat='Binary')

## Função objetivo

* Minimização dos custos normais e usando horas extras de produção, além da estocagem de camisetas: $\min \sum_{t \in T} c_{t}x_{t}$ + $\sum_{t \in \{1, 2\}} c_{t*}y_{t}$ + $\sum_{t \in \{1, 2, 3\}} h_{t}I_{t}$


In [5]:
modelo += pulp.lpSum([custos_de_producao[periodo] * producao_normal_vars[periodo] for periodo in periodos]) + \
pulp.lpSum([custos_de_producao_horas_extras[periodo_hora_extra] * producao_horas_extras_vars[periodo_hora_extra] for periodo_hora_extra in periodos_horas_extras]) + \
pulp.lpSum([custo_de_estoque[periodo] * estoque_vars[periodo] for periodo in [1, 2, 3]])

## Restrições

* Atendimento de demanda e definição de estoque: $x_{t} + y_{t} + I_{t-1} - d_{t} = I_{t}, \forall t \in \{1, 2\}$ e $x_{t} + I_{t-1} - d_{t} = I_{t}, \forall t \in \{3, 4\}$.

In [6]:
for periodo in periodos:
  if periodo == 1 or periodo == 2:
    if periodo == 1:
      modelo += producao_normal_vars[periodo] + producao_horas_extras_vars[periodo] - demanda[periodo] == estoque_vars[periodo]
    else:
      modelo += producao_normal_vars[periodo] + producao_horas_extras_vars[periodo] + estoque_vars[periodo - 1] - demanda[periodo] == estoque_vars[periodo]
  else:
    modelo += producao_normal_vars[periodo] + estoque_vars[periodo - 1] - demanda[periodo] == estoque_vars[periodo]

* Ao contratar horas extras, a fabricação de camisetas é limitada em 10.000 sse $b_{t} = 1$, matematicamente: $y_{t} \le 10000 * b_{t}, \forall t \in \{1, 2\}$

In [7]:
for periodo_hora_extra in periodos_horas_extras:
  modelo += producao_horas_extras_vars[periodo_hora_extra] <= 10000 * tem_horas_extras[periodo_hora_extra]

* A produção sem horas extras deve respeitar a capacidade máxima de produção da fábrica: $x_{t} \le capacidade\_maxima, \forall t \in T$.

In [8]:
for periodo in periodos:
  modelo += producao_normal_vars[periodo] <= capacidade_maxima

* Não deve haver estoque na última semana: $I_{4}=0$

Observação: requisito implícito. Supõe-se que a fábrica não desejaria estocar camisetas após o fim do campeonato.

In [9]:
modelo += estoque_vars[4] == 0

## Resolvendo o problema

* Mostrando os modelos (debug)

In [10]:
modelo

fabrica_de_camisetas:
MINIMIZE
0.2*estoque_1 + 0.2*estoque_2 + 0.2*estoque_3 + 2.0*producao_1 + 2.0*producao_2 + 2.5*producao_3 + 2.5*producao_4 + 2.8*producao_horas_extras_1 + 2.8*producao_horas_extras_2 + 0.0
SUBJECT TO
_C1: - estoque_1 + producao_1 + producao_horas_extras_1 = 5000

_C2: estoque_1 - estoque_2 + producao_2 + producao_horas_extras_2 = 10000

_C3: estoque_2 - estoque_3 + producao_3 = 30000

_C4: estoque_3 - estoque_4 + producao_4 = 60000

_C5: producao_horas_extras_1 - 10000 tem_horas_extras_1 <= 0

_C6: producao_horas_extras_2 - 10000 tem_horas_extras_2 <= 0

_C7: producao_1 <= 25000

_C8: producao_2 <= 25000

_C9: producao_3 <= 25000

_C10: producao_4 <= 25000

_C11: estoque_4 = 0

VARIABLES
0 <= estoque_1 Integer
0 <= estoque_2 Integer
0 <= estoque_3 Integer
0 <= estoque_4 Integer
0 <= producao_1 Integer
0 <= producao_2 Integer
0 <= producao_3 Integer
0 <= producao_4 Integer
0 <= producao_horas_extras_1 Integer
0 <= producao_horas_extras_2 Integer
0 <= tem_horas_extr

* Resolvendo o modelo

In [11]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [12]:
print('Status:', pulp.LpStatus[status])
print('Função objetivo:', f'A fábrica gastaria R${modelo.objective.value()} no melhor caso.')

print('\n\t\tSoluções')
for producao in producao_normal_vars:
  print(f'Quantidade de camisetas produzidas na semana {producao}:', producao_normal_vars[producao].value())
print()

for producao_hora_extra in producao_horas_extras_vars:
  print(f'Quantidade de camisetas produzidas na semana {producao_hora_extra} (contratando horas extras):', producao_horas_extras_vars[producao_hora_extra].value())
print()

for estoque in estoque_vars:
  print(f'Quantidade de camisetas estocadas na semana {estoque}:', estoque_vars[estoque].value())

Status: Optimal
Função objetivo: A fábrica gastaria R$258000.0 no melhor caso.

		Soluções
Quantidade de camisetas produzidas na semana 1: 25000.0
Quantidade de camisetas produzidas na semana 2: 25000.0
Quantidade de camisetas produzidas na semana 3: 25000.0
Quantidade de camisetas produzidas na semana 4: 25000.0

Quantidade de camisetas produzidas na semana 1 (contratando horas extras): 0.0
Quantidade de camisetas produzidas na semana 2 (contratando horas extras): 5000.0

Quantidade de camisetas estocadas na semana 1: 20000.0
Quantidade de camisetas estocadas na semana 2: 40000.0
Quantidade de camisetas estocadas na semana 3: 35000.0
Quantidade de camisetas estocadas na semana 4: 0.0
